In [8]:
import pandas as pd
import numpy as np
import tarfile
import io
from io import BytesIO
from bz2 import BZ2File


In [9]:
df = pd.read_csv('../../BLUED/location_001_dataset_001/location_001_eventslist.txt','rb',engine='python')
df

,"Timestamp,Label,Phase"
0,"10/20/2011 12:22:01.473,111,A"
1,"10/20/2011 12:37:40.507,111,A"
2,"10/20/2011 13:23:55.390,111,A"
3,"10/20/2011 13:39:08.157,111,A"
4,"10/20/2011 14:25:51.473,111,A"
...,...
150,"10/20/2011 22:39:43.040,128,B"
151,"10/20/2011 22:39:48.573,128,B"
152,"10/20/2011 22:39:52.223,128,B"
153,"10/20/2011 22:40:06.390,156,A"


In [10]:
import glob
data = []
from itertools import islice  
#path=r'BLUED/location_001_dataset_001'
path=r'../BLUED/test'
allFiles = glob.glob(path+"/location_001_ivdata_*.txt")
total = pd.DataFrame()
for file in allFiles:
    print(file)
    input_file = open(file) 
    for line in islice(input_file, 22, None): 
        data.append(line.split(','))
    graph = pd.DataFrame(data[1:],columns=data[0][0:4])
    data=[]
    #total = total.append(graph)
    total = pd.concat([total,graph])

total['VoltageA'] = total['VoltageA'].str.rstrip()
total =total.applymap(float)
total = total.abs()
total.reset_index(drop=True, inplace=True)
total

../BLUED/test\location_001_ivdata_001.txt
../BLUED/test\location_001_ivdata_002.txt
../BLUED/test\location_001_ivdata_003.txt


,X_Value,Current A,Current B,VoltageA
0,0.000000,0.742134,1.340304,149.180629
1,0.000083,0.933177,1.443007,144.194623
2,0.000167,0.827042,0.915865,138.956161
3,0.000250,0.911950,0.726094,136.810285
4,0.000333,1.102993,1.000208,132.581647
...,...,...,...,...
3685195,307.099583,1.251582,1.253236,47.630203
3685196,307.099667,1.251582,1.211065,42.612640
3685197,307.099750,1.251582,1.147808,37.721305
3685198,307.099833,1.272809,1.147808,32.672185


In [13]:
#处理data文件，去除'\n',','等符号，进行格式将str转化为float型，并将向量转为标量
data = []
from itertools import islice  
input_file = open("../BLUED/location_001_dataset_001/location_001_ivdata_001.txt") 
for line in islice(input_file, 22, None): 
    data.append(line.split(','))
graph = pd.DataFrame(data[1:],columns=data[0][0:4])
graph['VoltageA'] = graph['VoltageA'].str.rstrip()
graph =graph.applymap(float)
graph = graph.abs()
graph

,X_Value,Current A,Current B,VoltageA
0,0.000000,0.742134,1.340304,149.180629
1,0.000083,0.933177,1.443007,144.194623
2,0.000167,0.827042,0.915865,138.956161
3,0.000250,0.911950,0.726094,136.810285
4,0.000333,1.102993,1.000208,132.581647
...,...,...,...,...
1249195,104.099583,0.190231,4.563690,170.333996
1249196,104.099667,0.105323,4.163062,170.365553
1249197,104.099750,0.020415,3.720262,170.207768
1249198,104.099833,0.041642,3.087691,169.923755


In [14]:
#初始化datetime类型，适应采集率12KHz
from datetime import datetime
beg_time = datetime(2011,10,20,11,58,32,623499)
print(beg_time)

2011-10-20 11:58:32.623499


In [15]:
import datetime
delta=datetime.timedelta(microseconds=83*12000)
beg_time = beg_time+delta
print(beg_time)

2011-10-20 11:58:33.619499


In [16]:
#1s采集12000次，将时间序列以1s为单位：
N=12000
s = graph.groupby(graph.index // N).sum()
s

,X_Value,Current A,Current B,VoltageA
0,5999.50,12204.329000,26066.910742,1.325903e+06
1,17999.50,12215.476469,26088.452180,1.325630e+06
2,29999.50,12207.369450,26045.039459,1.325606e+06
3,41999.50,12202.377704,26055.480317,1.324704e+06
4,53999.50,12206.713568,26073.127798,1.324950e+06
...,...,...,...,...
100,1205999.50,12207.800253,26043.513865,1.325272e+06
101,1217999.50,12193.937632,26037.396246,1.325094e+06
102,1229999.50,12208.651158,26064.644476,1.325101e+06
103,1241999.50,12192.181881,26052.396333,1.324467e+06


In [17]:
#取均值，赋日期
s['Current A'] = s['Current A']/12000
s['Current B'] = s['Current B']/12000
s['VoltageA'] = s['VoltageA']/12000
s['X_Value'] = beg_time+s.index*delta
s

,X_Value,Current A,Current B,VoltageA
0,2011-10-20 11:58:33.619499,1.017027,2.172243,110.491954
1,2011-10-20 11:58:34.615499,1.017956,2.174038,110.469162
2,2011-10-20 11:58:35.611499,1.017281,2.170420,110.467162
3,2011-10-20 11:58:36.607499,1.016865,2.171290,110.391964
4,2011-10-20 11:58:37.603499,1.017226,2.172761,110.412512
...,...,...,...,...
100,2011-10-20 12:00:13.219499,1.017317,2.170293,110.439354
101,2011-10-20 12:00:14.215499,1.016161,2.169783,110.424464
102,2011-10-20 12:00:15.211499,1.017388,2.172054,110.425071
103,2011-10-20 12:00:16.207499,1.016015,2.171033,110.372283
